# VacationPy
----

The purpose of this project is take the list of 500+ cities selected in VacationPy and screen them for ideal vacation spots. First, the Google Maps API is used to generate a heat map of relative humidity using the city points. Second, weather conditions are screened to select a small subset (15-20) cities with ideal weather conditions for a vacation. The Google Maps API is used to identify a hotel within a specified search distance of each city's coordinates, and location pins are placed on the relative humidity heat map. When a pin is clicked, the hotel name and the city/country are shown.

Some notes:
1. I experimented with alternate color gradients for the heat map to show a larger range of colors. The ramp I selected uses rainbow from purple (low weights) to orange (high weights). I decided against a ramp with red since it was difficult to see the markers against the background.
2. I used the following "ideal weather conditions" to narrow down the city list to 17 candidates:
   * Dew point < 62<sup>o</sup> F
   * Maximum daily temperature between 70<sup>o</sup> F and 80<sup>o</sup> F
   * Minimum daily temperature > 50<sup>o</sup> F
   * Percent cloud cover < 30%
3. When searching for hotels, I had to increase the search radius to 30,000 meters (i.e., 30 km or about 19 mi) to find enough cities for the final map. I also used the `business_status` field from the query response to identify hotels that are open (since many are temporarily or permanently closed as a result of the pandemic).
4. Of the 17 candidate cities, 13 had at least one hotel within 30 km.
5. I used the pycountry module (https://pypi.org/project/pycountry/) to map country codes to country names which I used in the marker pin information boxes.


In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import json
import gmaps
import pycountry
from api_keys import g_key

## Import WeatherPy results into DataFrame

In [2]:
# Import results from CSV, check data types, and show DataFrame summary
df_weath = pd.read_csv('../WeatherPy/output/results.csv')
print(df_weath.dtypes)
display(df_weath.describe())
df_weath


name           object
country        object
lat           float64
lon           float64
clouds        float64
dew_point     float64
humidity      float64
max_temp      float64
min_temp      float64
wind_speed    float64
dtype: object


,lat,lon,clouds,dew_point,humidity,max_temp,min_temp,wind_speed
count,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000
mean,24.508478,17.513917,50.502000,54.475833,62.062500,81.242000,62.386333,6.997000
std,28.062465,79.978139,33.987402,15.044154,21.496533,14.279041,13.071095,4.532239
min,-42.477800,-165.103700,0.000000,2.600000,9.900000,37.400000,20.400000,1.100000
25%,2.325250,-59.299475,18.575000,43.875000,45.175000,73.375000,54.575000,3.800000
50%,32.641200,29.925250,50.200000,55.400000,64.900000,83.000000,64.100000,5.600000
75%,47.371500,85.954075,81.900000,67.200000,79.600000,91.300000,71.825000,9.025000
max,75.633400,173.528000,100.000000,81.200000,98.800000,122.000000,97.100000,32.400000


,name,country,lat,lon,clouds,dew_point,humidity,max_temp,min_temp,wind_speed
0,Ahonglukumu,CN,39.1937,77.3260,63.0,43.5,19.2,102.2,82.6,9.9
1,Akbulak,RU,51.0020,55.6174,23.8,44.2,37.8,81.7,59.3,8.9
2,Sanmeng,CN,22.8912,102.4106,97.4,68.3,93.9,77.6,65.2,1.7
3,Yabebyry,PY,-27.3793,-57.1654,1.2,25.2,43.3,55.9,38.1,8.9
4,Canindé,BR,-4.3589,-39.3117,28.8,64.1,64.2,96.3,69.3,5.0
...,...,...,...,...,...,...,...,...,...,...
595,Manziliya,IQ,32.4036,47.0039,0.0,37.0,10.2,118.7,97.1,23.7
596,Ksenyevka,RU,53.5598,118.7341,63.9,57.3,90.6,67.3,54.1,2.7
597,Yulin,CN,40.9925,125.9376,91.1,73.1,77.5,96.2,72.2,2.3
598,Dehi,AF,35.9571,67.2694,0.0,39.2,24.6,91.2,69.3,6.5


## Generate Humidity Heatmap

In [3]:
# Rainbow colors: red(low) --> purple(high)
gradient1 = ['rgba(0, 255, 255, 0)','rgba(255, 0, 0, 1)','rgba(255, 140, 0, 1)','rgba(255, 255, 0, 1)',
           'rgba(0, 255, 127, 1)','rgba(30, 144, 255, 1)', 'rgba(106, 90, 205, 1)']

# Rainbow colors: purple(low) --> red(high)
gradient2 = ['rgba(0, 255, 255, 0)','rgba(106, 90, 205, 1)','rgba(30, 144, 255, 1)','rgba(0, 255, 127, 1)',
           'rgba(255, 255, 0, 1)','rgba(255, 140, 0, 1)', 'rgba(255, 0, 0, 1)']

# Rainbow colors: purple(low) --> orange(high)
gradient3 = ['rgba(0, 255, 255, 0)','rgba(106, 90, 205, 1)','rgba(30, 144, 255, 1)','rgba(0, 255, 127, 1)',
           'rgba(255, 255, 0, 1)','rgba(255, 140, 0, 1)']

# Set API key
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = df_weath[["lat", "lon"]]

# Set relative humidity as weighting factor
weight = df_weath["humidity"]

# Plot Heatmap
fig = gmaps.figure(layout={'width':'980px', 'height':'600px'}, zoom_level=2, center=(30,0))

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weight, dissipating=False, max_intensity=100,
                                 point_radius=3.5, opacity=0.8, gradient=gradient3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='600px', width='980px'))

## Screen Cities Using "Ideal Weather Condtions"

In [4]:
# Set condtions for df.query()
cond = ['dew_point < 62',
        'max_temp < 80',
        'max_temp > 70',
        'min_temp > 50',
        'clouds < 30']

# Create and display resulting DataFrame
df_ideal = df_weath.query(' & '.join(cond)).copy().reset_index(drop=True)
df_ideal

,name,country,lat,lon,clouds,dew_point,humidity,max_temp,min_temp,wind_speed
0,Androhipano,MG,-24.8787,44.0500,20.5,57.9,67.7,77.2,63.5,17.4
1,Tsetseg,MN,46.5908,93.2684,22.9,39.2,32.7,79.3,59.0,11.4
2,Thunder Beach,CA,44.7966,-80.0630,27.0,61.7,83.3,78.1,58.9,5.8
3,Khoonkhwuttunne (historical),US,41.9484,-124.2031,5.9,50.2,64.5,71.6,56.7,1.1
4,Arrondissement de Châtellerault,FR,46.8333,0.3333,28.8,11.3,28.0,74.1,57.4,10.1
5,Welbourne Hill,AU,-27.3557,134.0831,10.7,37.5,39.8,72.3,53.4,10.7
6,Tahilt,MN,45.3466,96.6457,17.1,37.0,34.8,73.5,57.4,10.9
7,Niğde,TR,37.8333,34.7500,1.0,42.1,49.6,76.4,50.9,16.3
8,Marathon,US,42.4417,-76.0321,16.7,60.7,83.8,74.0,59.4,5.3
9,Тамбичозеро,RU,62.6696,36.1050,27.8,48.6,55.8,76.7,50.1,5.3


## Identify a Hotel Within Specified Search Distance for Each City

In [5]:
# Add country name field using pycountry
df_ideal['country_name'] = ''
for i, row in df_ideal.iterrows():
    country = pycountry.countries.get(alpha_2=row['country'])
    df_ideal.at[i,'country_name'] = country.name

# Add empty columns for hotel name and operational status
df_ideal['hotel_status'] = ''
df_ideal['hotel_name'] = ''

# Set static parameters for Nearby Place Search
place_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
dist_m = 30000
params = {
    'radius': dist_m,
    'type': 'lodging',
    'keyword': 'hotel',
    'language': 'en',
    'key': g_key
}

# Execute search
for i, row in df_ideal.iterrows():
    coords = str(row.lat) + ',' + str(row.lon)
    params['location'] = coords
    response = requests.get(place_url, params=params).json()
    
    # If a hotel is found within 'dist_m', the 'results' field will be non-null
    if response['results']:
        
        # Using dict.get() as an alternative to trapping KeyErrors
        name = response['results'][0].get('name')
        status = response['results'][0].get('business_status')
        if status:
            df_ideal.at[i,'hotel_status'] = status
        else:
            df_ideal.at[i,'hotel_status'] = 'Unknown'
        if name:
            df_ideal.at[i,'hotel_name'] = name
        else:
            df_ideal.at[i,'hotel_name'] = 'Unknown'
        
        # print log
        print(f"Row {i} processed")
    
    # 'hotel_status' and 'hotel_name' are left empty and message sent to print log
    else:
        print(f"Row {i} query returned empty results")

# Save results to CSV and show DataFrame
df_ideal.to_csv('output/my_hotels.csv')
df_ideal

Row 0 processed
Row 1 processed
Row 2 processed
Row 3 processed
Row 4 processed
Row 5 query returned empty results
Row 6 query returned empty results
Row 7 processed
Row 8 processed
Row 9 processed
Row 10 processed
Row 11 processed
Row 12 query returned empty results
Row 13 query returned empty results
Row 14 processed
Row 15 processed
Row 16 processed


,name,country,lat,lon,clouds,dew_point,humidity,max_temp,min_temp,wind_speed,country_name,hotel_status,hotel_name
0,Androhipano,MG,-24.8787,44.0500,20.5,57.9,67.7,77.2,63.5,17.4,Madagascar,OPERATIONAL,Villa Milahehe Mitongoa
1,Tsetseg,MN,46.5908,93.2684,22.9,39.2,32.7,79.3,59.0,11.4,Mongolia,OPERATIONAL,Алтай зам ХХК Кемп
2,Thunder Beach,CA,44.7966,-80.0630,27.0,61.7,83.3,78.1,58.9,5.8,Canada,OPERATIONAL,All Tucked Inn
3,Khoonkhwuttunne (historical),US,41.9484,-124.2031,5.9,50.2,64.5,71.6,56.7,1.1,United States,OPERATIONAL,Ocean View Inn
4,Arrondissement de Châtellerault,FR,46.8333,0.3333,28.8,11.3,28.0,74.1,57.4,10.1,France,OPERATIONAL,Le Château de Marçay
5,Welbourne Hill,AU,-27.3557,134.0831,10.7,37.5,39.8,72.3,53.4,10.7,Australia,,
6,Tahilt,MN,45.3466,96.6457,17.1,37.0,34.8,73.5,57.4,10.9,Mongolia,,
7,Niğde,TR,37.8333,34.7500,1.0,42.1,49.6,76.4,50.9,16.3,Turkey,OPERATIONAL,Grand Hotel Niğde
8,Marathon,US,42.4417,-76.0321,16.7,60.7,83.8,74.0,59.4,5.3,United States,OPERATIONAL,Hope Lake Lodge & Conference Center
9,Тамбичозеро,RU,62.6696,36.1050,27.8,48.6,55.8,76.7,50.1,5.3,Russian Federation,OPERATIONAL,Guest House Potapych


## Optional Step: Load Previously Generated Results

This avoids incurring unneeded charges using Google API Nearby Search.

In [6]:
# Read results CSV
df_ideal = pd.read_csv('output/my_hotels.csv', index_col=0)
df_ideal

,name,country,lat,lon,clouds,dew_point,humidity,max_temp,min_temp,wind_speed,country_name,hotel_status,hotel_name
0,Androhipano,MG,-24.8787,44.0500,20.5,57.9,67.7,77.2,63.5,17.4,Madagascar,OPERATIONAL,Villa Milahehe Mitongoa
1,Tsetseg,MN,46.5908,93.2684,22.9,39.2,32.7,79.3,59.0,11.4,Mongolia,OPERATIONAL,Алтай зам ХХК Кемп
2,Thunder Beach,CA,44.7966,-80.0630,27.0,61.7,83.3,78.1,58.9,5.8,Canada,OPERATIONAL,All Tucked Inn
3,Khoonkhwuttunne (historical),US,41.9484,-124.2031,5.9,50.2,64.5,71.6,56.7,1.1,United States,OPERATIONAL,Ocean View Inn
4,Arrondissement de Châtellerault,FR,46.8333,0.3333,28.8,11.3,28.0,74.1,57.4,10.1,France,OPERATIONAL,Le Château de Marçay
5,Welbourne Hill,AU,-27.3557,134.0831,10.7,37.5,39.8,72.3,53.4,10.7,Australia,NaN,NaN
6,Tahilt,MN,45.3466,96.6457,17.1,37.0,34.8,73.5,57.4,10.9,Mongolia,NaN,NaN
7,Niğde,TR,37.8333,34.7500,1.0,42.1,49.6,76.4,50.9,16.3,Turkey,OPERATIONAL,Grand Hotel Niğde
8,Marathon,US,42.4417,-76.0321,16.7,60.7,83.8,74.0,59.4,5.3,United States,OPERATIONAL,Hope Lake Lodge & Conference Center
9,Тамбичозеро,RU,62.6696,36.1050,27.8,48.6,55.8,76.7,50.1,5.3,Russian Federation,OPERATIONAL,Guest House Potapych


In [ ]:
## Create Marker Layer for Hotels

In [7]:
# Marker box CSS template
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{hotel_name}</dd>
<dt>Location</dt><dd>{city}, {country_name}</dd>
</dl>
"""

# Grab needed fields
hotels = []
for i, row in df_ideal.loc[df_ideal['hotel_status'] =='OPERATIONAL'].iterrows():
    coords = (row['lat'], row['lon'])
    hotels.append({'location': coords,
                    'hotel_name': row['hotel_name'],
                    'city': row['name'],
                    'country_name': row['country_name']})

# Set parameters for marker layer
hotel_locations = [hotel['location'] for hotel in hotels]
hotel_info = [info_box_template.format(**hotel) for hotel in hotels]

# Generate marker layer and add to figure
marker_layer = gmaps.marker_layer(hotel_locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig


Figure(layout=FigureLayout(height='600px', width='980px'))